# 실습 07: 다중 OCR 앙상블 기법

[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/leecks1119/document_ai_lecture/blob/master/notebooks/Lab07_앙상블.ipynb)

## 🎯 학습 목표
- 여러 OCR 엔진 결과 결합
- Voting 방식으로 정확도 향상
- 앙상블 전략 수립

## ⏱️ 소요 시간: 35분
## 📊 난이도: ⭐⭐⭐⭐☆


In [ ]:
# 한글 폰트 설치 및 설정 (Colab용)
!apt-get install -y fonts-nanum fonts-nanum-coding fonts-nanum-extra
!fc-cache -fv

# matplotlib 한글 폰트 설정
import matplotlib.pyplot as plt
import matplotlib.font_manager as fm

font_path = '/usr/share/fonts/truetype/nanum/NanumGothic.ttf'
fontprop = fm.FontProperties(fname=font_path, size=10)
plt.rc('font', family=fontprop.get_name())
plt.rc('axes', unicode_minus=False)

print("✅ 한글 폰트 설정 완료!")


In [ ]:
# 환경 설정
!pip install -q git+https://github.com/leecks1119/document_ai_lecture.git
!pip install -q paddlepaddle paddleocr easyocr
!apt-get install -y tesseract-ocr tesseract-ocr-kor


In [ ]:
# 테스트 이미지 생성 (중간 난이도)
from PIL import Image, ImageDraw, ImageFont
import cv2
import numpy as np

# 한글 폰트 로드
try:
    font = ImageFont.truetype("C:\\Windows\\Fonts\\malgun.ttf", 22)
except:
    try:
        font = ImageFont.truetype("/usr/share/fonts/truetype/nanum/NanumGothic.ttf", 22)
    except:
        font = ImageFont.load_default()

img = Image.new('RGB', (700, 200), color='white')
draw = ImageDraw.Draw(img)
draw.text((30, 30), "계약번호: CT-2025-1234\n계약금액: 15,000,000원\n담당자: 홍길동", fill='black', font=font)

# 약간의 노이즈 추가
img_cv = cv2.cvtColor(np.array(img), cv2.COLOR_RGB2BGR)
noise = np.random.normal(0, 10, img_cv.shape).astype(np.uint8)
img_cv = cv2.add(img_cv, noise)
cv2.imwrite('test_doc.jpg', img_cv)

print("✅ 테스트 이미지 생성 완료!")


In [ ]:
# 3개 OCR 엔진으로 인식
import pytesseract
from paddleocr import PaddleOCR
import easyocr

img = cv2.imread('test_doc.jpg')

# 1. Tesseract
text_tess = pytesseract.image_to_string(img, lang='kor+eng')

# 2. PaddleOCR (최신 API)
paddle_ocr = PaddleOCR(use_textline_orientation=True, lang='korean')
result_paddle = paddle_ocr.predict('test_doc.jpg')
text_paddle = ' '.join([line[1] for line in result_paddle[0]])

# 3. EasyOCR
easy_reader = easyocr.Reader(['ko', 'en'], gpu=False)
result_easy = easy_reader.readtext('test_doc.jpg')
text_easy = ' '.join([item[1] for item in result_easy])

print("="*60)
print("📊 개별 OCR 엔진 결과")
print("="*60)
print(f"\n1️⃣ Tesseract:\n{text_tess}")
print(f"\n2️⃣ PaddleOCR:\n{text_paddle}")
print(f"\n3️⃣ EasyOCR:\n{text_easy}")
print("="*60)


In [ ]:
# 단어 단위 Voting 앙상블
from collections import Counter

def voting_ensemble(texts, weights=None):
    """다수결 투표 방식"""
    if weights is None:
        weights = [1] * len(texts)
    
    # 모든 텍스트를 단어로 분리
    all_words = []
    for text, weight in zip(texts, weights):
        words = text.split()
        all_words.extend([(word, weight) for word in words])
    
    # 가중 투표
    word_votes = Counter()
    for word, weight in all_words:
        word_votes[word] += weight
    
    # 가장 많이 등장한 단어들로 재구성
    final_text = ' '.join([word for word, _ in word_votes.most_common()])
    
    return final_text, word_votes

# 앙상블 실행 (PaddleOCR과 EasyOCR에 더 높은 가중치)
ensemble_result, votes = voting_ensemble(
    [text_tess, text_paddle, text_easy],
    weights=[1, 2, 2]  # Paddle과 Easy에 2배 가중치
)

print("\n🎯 앙상블 결과:")
print("="*60)
print(ensemble_result)
print("="*60)
print("\n✅ 여러 엔진의 강점을 결합했습니다!")


In [ ]:
# 정확도 비교 (Ground Truth 기준)
import Levenshtein

ground_truth = "계약번호: CT-2025-1234 계약금액: 15,000,000원 담당자: 홍길동"

def calc_accuracy(pred, truth):
    distance = Levenshtein.distance(pred, truth)
    return max(0, (1 - distance / max(len(pred), len(truth))) * 100)

acc_tess = calc_accuracy(text_tess, ground_truth)
acc_paddle = calc_accuracy(text_paddle, ground_truth)
acc_easy = calc_accuracy(text_easy, ground_truth)
acc_ensemble = calc_accuracy(ensemble_result, ground_truth)

print("\n📊 정확도 비교:")
print("="*60)
print(f"  Tesseract:  {acc_tess:.2f}%")
print(f"  PaddleOCR:  {acc_paddle:.2f}%")
print(f"  EasyOCR:    {acc_easy:.2f}%")
print(f"  🎯 앙상블:   {acc_ensemble:.2f}%")
print("="*60)

improvement = acc_ensemble - max(acc_tess, acc_paddle, acc_easy)
if improvement > 0:
    print(f"\n✅ 앙상블로 +{improvement:.2f}%p 개선!")
else:
    print(f"\n💡 이 경우 단일 엔진이 더 우수합니다.")
